In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import init
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from SSIM_PIL import compare_ssim
from PIL import Image

In [3]:
image2=plt.imread(r'C:\Users\rohan\Downloads\Miniproject (1)\Miniproject\data\image\butterfly.jpg')
image1=plt.imread(r'C:\Users\rohan\Downloads\Miniproject (1)\Miniproject\data\image\fox.jpg')

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [5]:
def psnr(est,gt):
    psnr =-20*np.log10(np.linalg.norm((gt-est).reshape(-1))/(np.sqrt(gt.size)*np.max(gt)))
    return psnr

In [6]:
def fourier_features(pixels,k):
    b = np.random.normal(size=(k, 2))
    x=np.cos(2 * np.pi * np.dot(pixels, b.T))
    y=np.sin(2 * np.pi * np.dot(pixels, b.T))
    return np.concatenate((x, y), axis=1)

In [7]:
print(image1.shape)
x,y=np.meshgrid(np.linspace(0,1,512),np.linspace(0,1,512))
x=x.reshape((512*512),1)
y=y.reshape((512*512),1)
pixelold=np.hstack((x,y))
pixels=fourier_features(pixelold,64)
print(pixels.shape)
pixels=torch.Tensor(pixels)
transforms=torchvision.transforms.ToTensor()
image1=transforms(image1)
print(pixels.numpy().shape)


(512, 512, 3)
(262144, 128)
(262144, 128)


c:\Users\rohan\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\transforms\functional.py:149: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:205.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


In [8]:
image1=np.transpose(image1,(1,2,0))
print(image1.shape)

torch.Size([512, 512, 3])


In [9]:
class Implicit(nn.Module):
    def __init__(self):
        super(Implicit, self).__init__()
        self.relu1=nn.ReLU()
        self.linear1=nn.Linear(128,256)
        self.linear2=nn.Linear(256,512)
        self.linear3=nn.Linear(512,512)
        self.linear4=nn.Linear(512,256)
        self.linear5=nn.Linear(256,128)
        self.linear6=nn.Linear(128,3)
        self.act=nn.Sigmoid()
    def forward(self,x):
        x=self.linear1(x)
        x=self.relu1(x)
        x=self.linear2(x)
        x=self.relu1(x)
        x=self.linear3(x)
        x=self.relu1(x)
        x=self.linear4(x)
        x=self.relu1(x)
        x=self.linear5(x)
        x=self.relu1(x)
        x=self.linear6(x)
        x=self.act(x)
        return x

In [10]:
implicit=Implicit()
implicit.to(device)
image1.to(device)
criterion = nn.MSELoss() 
optimizer = torch.optim.Adam(implicit.parameters(),lr = 0.001)
n_epochs=2000
train_loss = 0
epochtrack=[]
psnrtrack=[]
simtrack=[]
x=pixels

In [10]:
for epoch in range(n_epochs):
    optimizer.zero_grad()
    out=implicit(x)
    out=out.reshape(512,512,3)
    loss=criterion(out,image1)
    loss.backward()
    optimizer.step()
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, loss.item()))
    psnrvalue=psnr(out.detach().numpy(),image1.detach().numpy())
    psnrtrack.append(psnrvalue)
    value = compare_ssim(Image.fromarray((out.detach().numpy()*255).astype(np.uint8)),Image.fromarray((image1.detach().numpy()*255).astype(np.uint8)))
    simtrack.append(value)

Epoch: 0 	Training Loss: 0.080791
No module named 'pyopencl'
Epoch: 1 	Training Loss: 0.079355
No module named 'pyopencl'
Epoch: 2 	Training Loss: 0.077501
No module named 'pyopencl'
Epoch: 3 	Training Loss: 0.074657
No module named 'pyopencl'
Epoch: 4 	Training Loss: 0.070143
No module named 'pyopencl'
Epoch: 5 	Training Loss: 0.063801
No module named 'pyopencl'
Epoch: 6 	Training Loss: 0.056421
No module named 'pyopencl'
Epoch: 7 	Training Loss: 0.049488
No module named 'pyopencl'
Epoch: 8 	Training Loss: 0.043256
No module named 'pyopencl'
Epoch: 9 	Training Loss: 0.039173
No module named 'pyopencl'
Epoch: 10 	Training Loss: 0.036546
No module named 'pyopencl'
Epoch: 11 	Training Loss: 0.035486
No module named 'pyopencl'
Epoch: 12 	Training Loss: 0.035075
No module named 'pyopencl'
Epoch: 13 	Training Loss: 0.034335
No module named 'pyopencl'
Epoch: 14 	Training Loss: 0.033341
No module named 'pyopencl'
Epoch: 15 	Training Loss: 0.032043
No module named 'pyopencl'
Epoch: 16 	Trainin

In [ ]:
print(psnrtrack)
print(simtrack)
plt.plot(psnrtrack)
plt.title('PSNR vs Epochs')
plt.show()
plt.plot(simtrack)
plt.title('SSIM vs Epochs')
plt.show()
out=implicit(pixels)
plt.imshow(out.reshape(512,512,3).detach().numpy())
plt.show()
plt.imshow(image1)
plt.show()
image1=image1.reshape(512*512*3)
out=out.reshape(512*512*3)
print(out.shape)
print(image1.shape)